### Modelos baseados em árvores

<br>

Ao longo do módulo, discutimos bastante as árvores de decisão, bem como ensemble de árvores, como Random Forest e algoritmos do tipo boosting.

Esses __ensembles acabam tendo muitos hiperparâmetros;__ escolhe-los de forma manual acaba sendo muito custoso e tedioso. 

Neste exercício, vamos discutir a respeito da metolodia __grid-search__, que otimiza essa busca de hiperparâmetros.

Considere o dataset abaixo (basta executar as células):

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [3]:
#problema de regressão

X, y = load_diabetes().data, load_diabetes().target
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.25, random_state = 42)
print(Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)

(331, 10) (111, 10) (331,) (111,)


Imagine que queremos testar - usando cross-validation - várias instâncias de Random Forests: com 10 árvores, com 100 árvores, com 1000 árvores, com profundidade máxima 1, 5, 10. 

Como podemos proceder? O código abaixo exemplifica um jeito:

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

for n_est in [10,100,100]:
    for prof in [1,5,10]:
        rf = RandomForestRegressor(n_estimators=n_est, max_depth=prof)
        cvres = cross_val_score(estimator=rf, X = Xtrain, y = ytrain, cv = 3, scoring='r2')
        print("estimators: ", n_est, " prof: ", prof, " | R2 mean / std: ", cvres.mean(), ' / ', cvres.std())

estimators:  10  prof:  1  | R2 mean / std:  0.3073527584006498  /  0.05367676802317831
estimators:  10  prof:  5  | R2 mean / std:  0.35716487589739776  /  0.050346487468848
estimators:  10  prof:  10  | R2 mean / std:  0.36441345393822955  /  0.0658401821678988
estimators:  100  prof:  1  | R2 mean / std:  0.33912205105022974  /  0.03453815766668794
estimators:  100  prof:  5  | R2 mean / std:  0.41886862287159293  /  0.05065032751620956
estimators:  100  prof:  10  | R2 mean / std:  0.413493423080362  /  0.04284494441613815
estimators:  100  prof:  1  | R2 mean / std:  0.3376274988442103  /  0.03704263555294598
estimators:  100  prof:  5  | R2 mean / std:  0.4336068528344921  /  0.032693290429567784
estimators:  100  prof:  10  | R2 mean / std:  0.4163140964067211  /  0.03708762626446199


Podemos, com algum trabalho, escolher o melhor modelo.

Se quisermos testar mais parâmetros, podemos aumentar nosso loop... mais isso vai ficando cada vez mais complicado.

A proposta do __grid-search__ é justamente fazer isso de forma mais automática!

Podemos importar a função GridSearchCV do módulo model_selection do sklearn e usá-la para isso. 
Na prática, precisamos definir um __estimador base__ para o grid. Além disso, precisamos definir um __dicionário de parâmetros__ a ser testado. Ainda, definiremos a quantidade de folds para cross-validation e qual a métrica de performance que queremos otimizar:

In [5]:
#importando a função
from sklearn.model_selection import GridSearchCV

In [6]:
#definindo o estimador base
estimador_base = RandomForestRegressor()

#definindo o dicionario de parâmetros do modelo
params_RF = {"n_estimators":[10,1000], "max_depth":[2,10]}

In [7]:
grid = GridSearchCV(estimator = estimador_base, 
                    param_grid = params_RF, 
                    scoring = 'r2', 
                    cv = 3)

grid

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

In [8]:
#treinando os modelos no grid
grid.fit(Xtrain, ytrain)

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

O objeto "grid", após o treinamento acima, conterá várias informações muito relevantes. 

__1- "best_params_":__ retorna os melhores parâmetros, de acordo com a métrica de performance avaliada na cross-validation;

__1- "best_score_":__ retorna o melhor score - métrica de performance - nos dados de validação;

__1- "best_estimator_":__ retorna o melhor modelo, já treinado;

__1- "cv_results_":__ retorna uma visão geral dos resultados.

In [9]:
grid.best_params_

{'max_depth': 10, 'n_estimators': 1000}

In [10]:
grid.best_score_

0.4252262947018622

In [11]:
grid.best_estimator_

RandomForestRegressor(max_depth=10, n_estimators=1000)

In [12]:
grid.cv_results_

{'mean_fit_time': array([0.01429574, 1.36269665, 0.01927336, 2.31513389]),
 'std_fit_time': array([0.00047126, 0.01460782, 0.00047599, 0.19300287]),
 'mean_score_time': array([0.00133006, 0.06548309, 0.00166257, 0.09075864]),
 'std_score_time': array([0.00046991, 0.00092791, 0.00046907, 0.01340648]),
 'param_max_depth': masked_array(data=[2, 2, 10, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[10, 1000, 10, 1000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'n_estimators': 10},
  {'max_depth': 2, 'n_estimators': 1000},
  {'max_depth': 10, 'n_estimators': 10},
  {'max_depth': 10, 'n_estimators': 1000}],
 'split0_test_score': array([0.35959838, 0.34774152, 0.27550989, 0.37247532]),
 'split1_test_score': array([0.4522152 , 0.44735019, 0.3718583 , 0.45629526]),
 'split2_test_score': array([0.37052979, 0.4

__Exercício 1:__ Utilizando o dataset abaixo, faça um grid_search com KNN's, Random Forests e GradientBoostings e retorne o melhor modelo de cada tipo.

__Obs.:__ Lembre-se de fazer um pré-processamento nos dados!

In [13]:
#preco_mediano_das_casas é a variável target
df = pd.read_csv("preco_casas.csv")
print(df.shape)
df.head()

(20640, 10)


,longitude,latitude,idade_mediana_das_casas,total_comodos,total_quartos,populacao,familias,salario_mediano,preco_mediano_das_casas,proximidade_ao_mar
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,PERTO DA BAÍA
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,PERTO DA BAÍA
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,PERTO DA BAÍA
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,PERTO DA BAÍA
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,PERTO DA BAÍA


In [14]:
# dividindo os dados

dftrain, dftest = train_test_split(df, random_state = 0)
print(df.shape)
print(dftrain.shape)
print(dftest.shape)

(20640, 10)
(15480, 10)
(5160, 10)


In [15]:
# Pré-processamento dos dados

def preprocessamento_completo(df, dataset_de_treino = True, cat_encoder = None, std_scaler = None):

    dff = df.copy()

    #retirando valores faltantes
    dff = dff.dropna(axis = 0)
    
    variaveis_para_normalizar = ['latitude',
                                 'longitude',
                                'idade_mediana_das_casas',
                                 'total_comodos',
                                 'total_quartos',
                                 'populacao',
                                 'familias',
                                 'salario_mediano']

    if dataset_de_treino:  
        
        #OHE
        encoder = OneHotEncoder()
        df_prox_mar_OHE = encoder.fit_transform(dff[['proximidade_ao_mar']]).toarray()

        #normalização
        sc = StandardScaler()
        variaveis_norm = sc.fit_transform(dff[variaveis_para_normalizar])
        
        X, y =  np.c_[df_prox_mar_OHE, variaveis_norm], dff.preco_mediano_das_casas.values
        return X, y, encoder, sc
    
    else:
        #OHE
        df_prox_mar_OHE = cat_encoder.transform(dff[['proximidade_ao_mar']]).toarray()
        
        #normalização
        variaveis_norm = std_scaler.transform(dff[variaveis_para_normalizar]) 
        
        X, y =  np.c_[df_prox_mar_OHE, variaveis_norm], dff.preco_mediano_das_casas.values
        return X, y

In [16]:
Xtrain, ytrain, encoder_train, scaler_train  = preprocessamento_completo(df = dftrain, 
                                                                         dataset_de_treino = True, 
                                                                         cat_encoder = None, 
                                                                         std_scaler = None)

In [17]:
Xtrain.shape, ytrain.shape, dftrain.shape

((15331, 13), (15331,), (15480, 10))

In [18]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

In [19]:
# faça um grid_search com KNN's, Random Forests e GradientBoostings e retorne o melhor modelo de cada tipo:

#KNN

#definindo o estimador base
estimador_base = KNeighborsRegressor()

#definindo o dicionario de parâmetros do modelo
params_RF = {'n_neighbors': [3,5,11,19],
            'weights': ['uniform', 'distance']}

In [20]:
grid = GridSearchCV(estimator = estimador_base, 
                    param_grid = params_RF, 
                    verbose = 1,
                    cv = 3,
                    n_jobs = -1)

grid

GridSearchCV(cv=3, estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid={'n_neighbors': [3, 5, 11, 19],
                         'weights': ['uniform', 'distance']},
             verbose=1)

In [21]:
#treinando os modelos no grid
grid.fit(Xtrain, ytrain)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    8.6s finished


GridSearchCV(cv=3, estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid={'n_neighbors': [3, 5, 11, 19],
                         'weights': ['uniform', 'distance']},
             verbose=1)

In [22]:
grid.best_params_

{'n_neighbors': 11, 'weights': 'distance'}

In [23]:
grid.best_score_

0.724256628091983

In [24]:
grid.best_estimator_

KNeighborsRegressor(n_neighbors=11, weights='distance')

In [25]:
grid.cv_results_

{'mean_fit_time': array([0.17552988, 0.09474587, 0.09740424, 0.0997328 , 0.07779177,
        0.08011723, 0.07945363, 0.0787882 ]),
 'std_fit_time': array([0.01020479, 0.01345586, 0.01649633, 0.01199542, 0.0029352 ,
        0.00124439, 0.00248752, 0.00293558]),
 'mean_score_time': array([0.50930556, 0.49035557, 0.57812142, 0.57147185, 0.74367801,
        0.73004834, 0.9032522 , 0.90724063]),
 'std_score_time': array([0.0084752 , 0.05083682, 0.01852162, 0.01894092, 0.01141024,
        0.01389101, 0.02974666, 0.03912318]),
 'param_n_neighbors': masked_array(data=[3, 3, 5, 5, 11, 11, 19, 19],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_weights': masked_array(data=['uniform', 'distance', 'uniform', 'distance',
                    'uniform', 'distance', 'uniform', 'distance'],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=objec

In [26]:
# Random Forest

#definindo o estimador base
estimador_base = RandomForestRegressor()

#definindo o dicionario de parâmetros do modelo
params_RF = {"n_estimators":[10,1000], "max_depth":[2,10]}

In [27]:
grid = GridSearchCV(estimator = estimador_base, 
                    param_grid = params_RF, 
                    scoring = 'r2', 
                    cv = 3)

grid

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

In [28]:
#treinando os modelos no grid
grid.fit(Xtrain, ytrain)

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

In [29]:
grid.best_params_

{'max_depth': 10, 'n_estimators': 1000}

In [30]:
grid.best_score_

0.7794228501511725

In [31]:
grid.best_estimator_

RandomForestRegressor(max_depth=10, n_estimators=1000)

In [32]:
grid.cv_results_

{'mean_fit_time': array([ 0.120025  , 11.57172004,  0.43284289, 45.64238103]),
 'std_fit_time': array([0.01033335, 0.56974169, 0.00141171, 0.56450192]),
 'mean_score_time': array([0.00299311, 0.19380418, 0.0079782 , 0.770281  ]),
 'std_score_time': array([3.44402769e-06, 1.25028403e-02, 1.66324373e-06, 7.67853569e-02]),
 'param_max_depth': masked_array(data=[2, 2, 10, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[10, 1000, 10, 1000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'n_estimators': 10},
  {'max_depth': 2, 'n_estimators': 1000},
  {'max_depth': 10, 'n_estimators': 10},
  {'max_depth': 10, 'n_estimators': 1000}],
 'split0_test_score': array([0.51752273, 0.5196658 , 0.76956395, 0.7790996 ]),
 'split1_test_score': array([0.50342266, 0.5081175 , 0.75520933, 0.76921909]),
 'split2_test_score': ar

In [33]:
# GradientBoosting

#definindo o estimador base
estimador_base = GradientBoostingRegressor()

#definindo o dicionario de parâmetros do modelo
params_RF = {'n_estimators': [10,50,100], 'learning_rate':[0.001,0.01, 0.1]}

In [34]:
grid = GridSearchCV(estimator = estimador_base, 
                    param_grid = params_RF, 
                    scoring = 'r2', 
                    cv = 3)

grid

GridSearchCV(cv=3, estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.001, 0.01, 0.1],
                         'n_estimators': [10, 50, 100]},
             scoring='r2')

In [35]:
#treinando os modelos no grid
grid.fit(Xtrain, ytrain)

GridSearchCV(cv=3, estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.001, 0.01, 0.1],
                         'n_estimators': [10, 50, 100]},
             scoring='r2')

In [36]:
grid.best_params_

{'learning_rate': 0.1, 'n_estimators': 100}

In [37]:
grid.best_score_

0.7723931835788703

In [38]:
grid.best_estimator_

GradientBoostingRegressor()

In [39]:
grid.cv_results_

{'mean_fit_time': array([0.24201012, 1.24830047, 2.41489172, 0.22506372, 1.17650739,
        2.37963525, 0.22740428, 1.18249679, 2.42451262]),
 'std_fit_time': array([0.01030259, 0.04112947, 0.16892991, 0.00232576, 0.06029771,
        0.10138562, 0.0016437 , 0.05050331, 0.07130797]),
 'mean_score_time': array([0.00914423, 0.00897511, 0.01329549, 0.00199437, 0.00731333,
        0.01329851, 0.00231489, 0.00598423, 0.01196957]),
 'std_score_time': array([1.01077136e-02, 2.82007373e-03, 4.73012300e-04, 3.37174788e-07,
        4.70471221e-04, 4.70022300e-04, 4.78963574e-04, 2.97360213e-07,
        1.38284150e-03]),
 'param_learning_rate': masked_array(data=[0.001, 0.001, 0.001, 0.01, 0.01, 0.01, 0.1, 0.1, 0.1],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[10, 50, 100, 10, 50, 100, 10, 50, 100],
              mask=[False, False, False, False

__Exercício 2:__ Crie uma classe para comparar o grid_search dentre vários modelos distintos.
    
    
Essa classe, gridSearchAll(), já está pré-desenvolvida no código abaixo. O exercício consiste de __completar essa classe.__ Para isso, crie o métodos fit_all, que irá treinar, usando grid_search, todos os grids que tenham sido pré-construídos e inseridos na classe.
Ainda, a quantidade de folds para a validação cruzada no grid_search deve ser implementada no método construtor da classe, bem como qual a métrica de performance a ser avaliada. 
Finalmente, salve o melhor modelo de cada grid e tenha um método best_all_grid_models que retorna o melhor modelo dentre todos os grids.

In [80]:
class gridSearchAll():
    
    def __init__(self, scoring, num_folds):
        self.grid_models = []
        self.scoring = scoring
        self.num_folds = num_folds
        self.best = []
    
    def insert_model(self, estimator_base, param_grid):
        self.grid_models.append([estimator_base, param_grid])
        
    def fit_all(self, X, y):
        for estimator, param in self.grid_models:
            gridCV = GridSearchCV(estimator=estimator, param_grid=param, cv=self.num_folds, scoring=self.scoring)
            gridCV.fit(X, y)
            self.best.append(gridCV.best_params_)
        print('Modelo treinado!')
   
    def best_all_grid_models(self):
        best_models = []
        for gridCV in self.best:
            best_models.append(gridCV.best_estimator_)
        return best_models

In [74]:
gd = gridSearchAll(scoring='r2', num_folds=5)

In [ ]:
gd.grid_models

In [ ]:
params_RF

In [ ]:
gd.insert_model(estimator_base = RandomForestRegressor(), param_grid = params_RF)

In [ ]:
gd.grid_models

In [ ]:
gd.insert_model(estimator_base = KNeighborsRegressor(), param_grid = {"n_neighbors":[1,2,10]})

In [ ]:
gd.grid_models

__Exercício 3:__ Usando a classe criada, analise novamente os modelos criados no exercício 1.

In [75]:
rf = RandomForestRegressor()
knn = KNeighborsRegressor()
gb = GradientBoostingRegressor()

In [76]:
# os modelos serão inseridos um a um na classe utilizando o método insert_model

gd.insert_model(rf, [{'n_estimators': [10,50,100], 'max_depth':[2,8,12,20]}]) 
gd.insert_model(knn, [{'n_neighbors': [3,5,11,19], 'weights': ['uniform', 'distance']}])
gd.insert_model(gb, [{'n_estimators': [10,50,100], 'learning_rate':[0.001,0.01, 0.1]}])

In [ ]:
gd.grid_models

In [77]:
gd.fit_all(Xtrain, ytrain)

Modelo treinado!
